# 🎫 GeniusXP

## 💡 Explicação do projeto
Uma plataforma centralizada para gestão de eventos que simplifica operações como inscrições, pagamentos e check-in, enquanto aumenta o engajamento com enquetes e networking. A inteligência artificial da GeniusXP utiliza as preferências dos usuários para oferecer uma experiência altamente personalizada e otimizar o planejamento. Com análise de sentimento e assistência virtual, a plataforma proporciona interações mais significativas, elevando a eficiência da gestão e tornando os eventos mais impactantes para cada participante.

## 📖 Metodologia utilizada
A aplicação Bluetrip utiliza técnicas avançadas de inteligência artificial para personalizar a experiência do usuário e automatizar processos, integrando o framework TensorFlow com a biblioteca FastAPI para criar uma API robusta e eficiente. A seguir, descrevemos a metodologia empregada para desenvolver este componente da aplicação.

## 👥 Equipe
Este projeto está sendo desenvolvido pelos seguintes membros:

- RM99565 - Erick Nathan Capito Pereira (2TDSPV)
- RM99577 - Guilherme Dias Gomes (2TDSPX)
- RM550889 - Hemily Nara da Silva (2TDSPX)
- RM550841 - Lucas Araujo Oliveira Silva (2TDSPV)
- RM99409 - Michael José Bernardi Da Silva (2TDSPX)


## 🎲 Datasets utilizados

- [geniusxp-database-next-projects.xlsx](app/model/geniusxp-database-next-projects.xlsx)

## Configuração do Ambiente e Importações

In [ ]:
%pip install -U FastAPI uvicorn pandas pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

from pydantic import BaseModel

## Pré-processamento dos dados

In [4]:
# Carregar os dados do Excel
recommendations_df = pd.read_excel('geniusxp-database-next-projects.xlsx')

# Exibir os primeiros registros para ver o formato dos dados
print(recommendations_df.head())

                                      nome  \
0  Target - Planejamento de Marketing BAUC   
1                              Axion Nexus   
2                                  HelpCar   
3                                    Nodes   
4                                 Robô EVA   

                                           descricao  \
0  Nós da Target estamos ajudando a agência BAUC ...   
1  Axion Nexus é um jogo 3D de puzzle em primeira...   
2  A HelpCar é uma plataforma inovadora que utili...   
3  Fizemos um aplicativo focado em melhorar a for...   
4  A robocup é uma competição anual feita pelas t...   

                                   curso                                 tipo  \
0       Marketing Digital e Data Science                            Challenge   
1                         Jogos Digitais                            Challenge   
2  Análise e Desenvolvimento de Sistemas                            Challenge   
3                    Produção Multimídia                      

## Passo 2: Transformar os segmentos em uma matriz de características

In [5]:
# Criar um vetorizador que conte a frequência dos segmentos
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(';'))
segment_matrix = vectorizer.fit_transform(recommendations_df['segmentos'])

# Exibir os segmentos únicos
print(vectorizer.get_feature_names_out())

['agronegocio' 'alimentacao' 'banking' 'comercio' 'comunicacao' 'educacao'
 'entretenimento' 'governanca' 'ia-iot' 'jogos' 'moda' 'saude' 'seguranca'
 'sustentabilidade' 'transporte' 'turismo']


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Passo 3: Calcular a similaridade entre os interesses do usuário e os projetos

In [6]:
def recommend_projects(interests):
    # Transformar os interesses do usuário em um vetor
    user_interests_vec = vectorizer.transform([interests])

    # Calcular a similaridade de cosseno entre os interesses do usuário e os projetos
    similarity = cosine_similarity(user_interests_vec, segment_matrix)

    # Ordenar os projetos por similaridade
    recommended_projects_indexes = similarity.argsort()[0][::-1]

    # Retornar os projetos mais semelhantes
    return recommendations_df.iloc[recommended_projects_indexes]

# Exemplos de interesses do usuário
interesses_usuario = "ia-iot;entretenimento"

# Recomendar projetos
projetos_recomendados = recommend_projects(interesses_usuario)

# Exibir os projetos recomendados
print(projetos_recomendados[['nome', 'descricao', 'curso', 'tipo', 'segmentos', 'url', 'local', 'horario']][1:5])


         nome                                          descricao  \
8     Spot.It  Um jogo onde o seu objetivo é tirar selfies co...   
31     RPS-IA  O projeto utiliza visão computacional e inteli...   
4    Robô EVA  A robocup é uma competição anual feita pelas t...   
43  TechPeach  O TechPeach utiliza inteligência artificial e ...   

                                    curso  \
8   Análise e Desenvolvimento de Sistemas   
31                Inteligência Artificial   
4                  Sistemas de Informação   
43  Análise e Desenvolvimento de Sistemas   

                                   tipo              segmentos  \
8                  Iniciação Científica  entretenimento;ia-iot   
31                 Iniciação Científica  entretenimento;ia-iot   
4   Competições (ex: Robocup, CTF, etc)  entretenimento;ia-iot   
43                            Challenge  entretenimento;ia-iot   

                               url                              local  \
8   https://ic-spot-it.vercel

## Instanciando o FastAPI
Instanciação do FastAPI e configuração do middleware CORS para permitir comunicações com o front-end.

In [7]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

## Criação das rotas para conexão com o front-end

### Rota de status para verificar se a aplicação está sendo executada
#### [GET] /status

In [8]:
@app.get("/status")
async def app_status():
    return 'ok'

### Rota para obtenção de recomendações de acordo com os interesses do usuário
#### [GET] /projects-recommendation - interests: str

In [10]:
class Project(BaseModel):
    nome: str
    descricao: str
    curso: str
    tipo: str
    segmentos: str
    local: int | str
    horario: str

@app.get("/projects-recommendation")
async def get_recommendation(interests: str):
    projects = recommend_projects(interests).head(5)

    project_list = [Project(
        nome=row["nome"],
        descricao=row["descricao"],
        curso=row["curso"],
        tipo=row["tipo"],
        segmentos=row["segmentos"],
        local=row["local"],
        horario=row["horario"]
    ) for idx, row in projects.iterrows()]

    return project_list

## Execução do servidor HTTP do FastAPI utilizando Uvicorn

In [ ]:
if __name__ == "__main__":
    uvicorn.run("app:app", host="127.0.0.1", port=8000)